<a href="https://colab.research.google.com/github/hectic97/Boostcamp-AI-Tech/blob/main/U-stage/Week8/Knowledge_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 경량화 과제
경량화 세번째 과제 파일 입니다.

이번 과제는 지식증류방법(Knowledge distillation)을 활용하여 Student 모델 학습하고, alpha와 temperature에 따라 정확도가 어떻게 변화하는지 확인해보도록하겠습니다. 


세부 내용은 다음과 같습니다. 

```
과제3. 지식증류방법(Knowledge distillation)을 활용하여 Student 모델 학습
1. Teacher 모델정의
2. alpha 와 Temperature 변화시켜 정확도 측정 해보기  

```

In [ ]:
# GPU 할당확인 
import torch

print('CUDA GPU availalbe : {}'.format(torch.cuda.is_available()))
try:
    print('{} GPU(s) is(are) allocated'.format(torch.cuda.device_count()))
except:
    print('GPUs are not allocated. Current runtime is on CPU.')

## 드라이브 연결 및 PATH 설정 

드라이브 마운트와 PATH설정을 하는 단계입니다. 예제는 아래와 같은 상황에서 돌아가도록 맞춰졌으며, 
자신의 드라이브 환경에 맞추어 설정하시면 되겠습니다. 

```
dataset
  |
  |-  teacher.pth
  |-  경량화 과제_3(문제)
  |-  APY170401435_subset_13gb
```





In [ ]:
# 코랩 내 드라이브와 연결
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Path 설정 
import os

if not os.path.exists('/content/gdrive/My Drive/dataset/'):
  os.mkdir('/content/gdrive/My Drive/dataset/')
os.chdir('/content/gdrive/My Drive/dataset/') # Data_Path
current_path = os.getcwd() # current folder
print('current_path', current_path)

## 파일이동 확인 
정상적으로 파일이 옮겨졌는지 확인 합니다. 

In [ ]:
from glob import glob

image = sorted(glob('./total/*.jpg'))
meta = sorted(glob('./total/*.metadata'))

if len(image) == len(meta):
    print('train data ({} files) unzip succeed'.format(len(image)))
else:
    print('train data unzip failed')

## 사용자 정의함수 


In [ ]:
# import 
import numpy as np
from PIL import Image
import json
from glob import glob
from pathlib import Path

import cv2
import os
import random
import matplotlib.pyplot as plt
from PIL import Image
import imutils
import time
from datetime import timedelta
from collections import deque, defaultdict
import re 
import matplotlib.image as mpimg
from collections import OrderedDict
from skimage import io, transform
from math import *
import xml.etree.ElementTree as ET 


# torch 
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader

# Dataset / Dataloader


1. Image_Dataset_ : dataset 데이터셋을 정의하는 객체
2. get_loader : Dataloader 객체를 return하는 함수



In [ ]:
class AttributeDict(dict):
    def __init__(self):
        self.__dict__ = self
        

class ConfigTree:
    def __init__(self):
        self.DATASET = AttributeDict()
        self.SYSTEM = AttributeDict()
        self.TRAIN = AttributeDict()
        self.MODEL = AttributeDict()
        self.KD = AttributeDict()

class Image_Dataset_(Dataset):
    def __init__(self, root, cfg=None, transform=None):
        """
        root : './data/'
        """
        super(Image_Dataset_, self).__init__()
        self.root = root
        self.cfg = cfg
        self.transform = transform
        self.images = sorted(glob(self.root + '/*.jpg'))
        self.annotations = sorted(glob(self.root + '/*.metadata'))
        
    def __getitem__(self, idx):
# ---------------------------------------------------------------------------- #
#  1. getitem 완성시키기
# * 주의 * 밑에 사용된 get_gt, label2logit 함수를 이용하여 label 을 만들어야함.
#   1) .metadata를 get_gt로 읽기
#   2) label2logt을 활용하여 1) 값 int 로 변경 
# ---------------------------------------------------------------------------- #

       # 채우기 #


        if self.transform:
            image = self.transform(image)
        
        return image, label
        
    def __len__(self):
        assert len(self.images) == len(self.annotations), "# of image files and # of meatadata files do not match"
        return len(self.images)
    
     
# ---------------------------------------------------------------------------- #
#  과제 1 에서 작성한 함수를 적용
# .metadata 를 읽어서 22번째 값(male or female) 을 가져오기 ... 
# ---------------------------------------------------------------------------- #

    def get_gt(self, meta_file):
        gt_class = None
        meta_file = Path(meta_file)
        f = open(meta_file, "r")
        

        # 채우기 #


        return gt_class
    
    def label2logit(self,label):
        str_dict = {
            'female': 0,
            'male':1
        }
        return str_dict[label]

def get_loader(dataset, cfg, shuffle=True):
    return DataLoader(dataset, batch_size=cfg.TRAIN.BATCH_SIZE, shuffle=shuffle)

# Image Augmentation 
 get_augmentation : image transform 객체를 return하는 함수
- resize_crop 
- random_flip
- color_jitter


In [ ]:
def get_augmentation(size=224, use_flip=True, use_color_jitter=False, use_gray_scale=False, use_normalize=False):
    resize_crop = transforms.RandomResizedCrop(size=size)
    random_flip = transforms.RandomHorizontalFlip(p=0.5)
    color_jitter = transforms.RandomApply([
        transforms.ColorJitter(0.8, 0.8, 0.8, 0.2)
    ], p=0.8)
    
    gray_scale = transforms.RandomGrayscale(p=0.2)
    normalize = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    to_tensor = transforms.ToTensor()
    
    transforms_array = np.array([resize_crop, random_flip, color_jitter, gray_scale, to_tensor, normalize])
    transforms_mask = np.array([True, use_flip, use_color_jitter, use_gray_scale, True, use_normalize])
    
    transform = transforms.Compose(transforms_array[transforms_mask])
    
    return transform


## Student model





In [ ]:
# ---------------------------------------------------------------------------- #
# (코드 채우기) 과제 2에서 정의한 Student model 
# ---------------------------------------------------------------------------- #

class Student(nn.Module):
  def __init__(self):
      super().__init__()





    return x

## Train

- Trainer : config에서 주어진 실험 조건들로 학습을 행하는 객체

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import timedelta
import time
from tqdm import tqdm

class Trainer:
    def __init__(self, model, optimizer, cfg, device, train_loader, teacher=None, val_loader=None):
        self.cfg = cfg
        self.device = device
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.save_path = cfg.BASE_SAVE_DIR
        
        if teacher:
            self.teacher = teacher.to(device)
            self.temperature = cfg.KD.TEMPERATURE
            self.alpha = cfg.KD.ALPHA
        
        print("[INFO] using gpu {}".format(cfg.SYSTEM.GPU))

        self.model = model.to(device)
        
        update_params = [p for p in model.parameters() if p.requires_grad]
        self.optimizer = optimizer(update_params, lr=cfg.TRAIN.BASE_LR, weight_decay=cfg.TRAIN.WEIGHT_DECAY)
        self.criterion = nn.CrossEntropyLoss()
        self.total_step = 0
        self.training_time = 0
        
        print(model)
        print()
        print(optimizer)
        print()
        
            
    def KD_train_eval(self, name='KD_student'):
        print("training KD start!\n")
        for epoch in range(self.cfg.TRAIN.EPOCH):
            self.KD_train_one_epoch(epoch)
            self.test_one_epoch(epoch)
            self.save_model(epoch,name)
    
    def KD_train_one_epoch(self,epoch):
        print('\nEpoch: %d' % epoch)
        print(f'[INFO] Temperature {self.temperature}, Alpha {self.alpha}')
        self.model.train()
        self.teacher.eval()
        
        train_loss = 0
        correct = 0
        total = 0
        for batch_idx, (inputs, targets) in enumerate(self.train_loader):
            inputs, targets = inputs.to(self.device), targets.to(self.device)
            self.optimizer.zero_grad()
            
            outputs = self.model(inputs) # student
            teacher_outputs = self.teacher(inputs) # teacher 
            loss = self.loss_fn_kd(outputs, targets, teacher_outputs)
            loss.backward()
            self.optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            if batch_idx % self.cfg.TRAIN.PERIOD ==0:
                print('Loss: %.3f | Acc: %.3f%% ' %(train_loss/(batch_idx+1),100.*correct/total))
      
        
    def test_one_epoch(self,epoch):
        self.model.eval()
        
        test_loss = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(self.val_loader):
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()
                
        print(f'Accuracy of the network on the {total} test images: %d %%' % (
        100 * correct / total))
        
    # ---------------------------------------------------------------------------- #
    # (코드 채우기) Kd Loss 정의하기 
    # ---------------------------------------------------------------------------- #  
        
    def loss_fn_kd(self, outputs, labels, teacher_outputs):
        """
        params = T & alpha in `self`
        Compute the knowledge-distillation (KD) loss given outputs, labels.
        "Hyperparameters": temperature and alpha
        NOTE: the KL Divergence for PyTorch comparing the softmaxs of teacher
        and student expects the input tensor to be log probabilities
        """
        
        # 채우기 



        return KD_loss
        
    def save_model(self, epoch, name='KD'):
        print('saved model {}'.format(self.save_path)) 
        if not os.path.exists(self.save_path):
            os.mkdir(self.save_path)
        file_name = '{}_{}.pth'.format(name, epoch)
        file_path = os.path.join(self.save_path, file_name)
        torch.save(self.model.state_dict(), file_path)
            
        return None 
              

## Teacher Network

In [ ]:
class Teacher(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrain_extractor = self.freeze()

           
    def freeze(self):
        model = models.resnet50(pretrained=True)
        num_classes = 2
        in_features = model.fc.in_features
        model.fc = nn.Sequential(nn.Linear(in_features, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512,num_classes))
        
        return model 
        
    def forward(self, x):
        x = self.pretrain_extractor(x)
        return x

# Config

모델 변수(즉, 모델의 학습 조건을 설정해주는 공간 ) 

In [ ]:
# config
config = ConfigTree()
config.SYSTEM.GPU = 0 # GPU 번호 
config.BASE_SAVE_DIR = './outputs' # 모델 파라미터가 저장되는 위치

config.DATASET.ROOT = "./total" # 데이터 위치
config.DATASET.NUM_CLASSES = 2 # 분류해야 하는 클래스 종류의 수 
config.DATASET.RATIO = 0.3 # train, test split 비율 
config.DATASET.TEACHER_WEIGHT_PATH = './teacher_9.pth' # teacher weight 경로



# hyperparameter ... 
# 사용할 augmentation
config.TRAIN.AUGMENTATION = {'size' : 224,
                             'use_flip' : True,
                             'use_color_jitter' : False,
                             'use_normalize' : True
} 
config.TRAIN.WEIGHT_DECAY = 0.9
config.TRAIN.EPOCH = 30 # 총 학습 에폭 
config.TRAIN.BATCH_SIZE = 32 # 배치 사이즈 
config.TRAIN.BASE_LR = 1e-04 # 러닝 레이트 
config.TRAIN.PERIOD = 1 # loss 측정 주기
config.MODEL.OPTIM = 'SGD' # 'ASGD, Adam, rmsprop etc...'

config.TRAIN.TEACHER_MODEL = Teacher()
config.TRAIN.STUDENT_MODEL = Student()

## main 

학습 코드를 실행 

In [ ]:
def main(config):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  data_root = config.DATASET.ROOT
  transform = get_augmentation(**config.TRAIN.AUGMENTATION)
  dataset = Image_Dataset_(data_root, config, transform=transform)

  len_valid_set = int(config.DATASET.RATIO*len(dataset))
  len_train_set = len(dataset) - len_valid_set

  print("The length of Train set is {}".format(len_train_set))
  print("The length of Valid set is {}".format(len_valid_set))

  train_dataset , valid_dataset,  = torch.utils.data.random_split(dataset , [len_train_set, len_valid_set])

  # shuffle and batch the datasets
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config.TRAIN.BATCH_SIZE, shuffle=True,)
  valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=config.TRAIN.BATCH_SIZE, shuffle=True,)


  # Load Teacher dict 
  teacher = config.TRAIN.TEACHER_MODEL
  weight_path = config.DATASET.TEACHER_WEIGHT_PATH 

  teacher.load_state_dict(torch.load(weight_path,map_location='cuda:0'))
  for param in teacher.parameters():
      param.requires_grad = False
      
  teacher.to(device)


  student = config.TRAIN.STUDENT_MODEL
  student.to(device)
  optimizer = optim.__dict__[config.MODEL.OPTIM]


  if config.KD.ON_OFF == 'on':
    trainer = Trainer(student, optimizer, config, device, train_loader, teacher=teacher, val_loader=valid_loader)
    trainer.KD_train_eval()
  else:
    pass

## Student Network Knowledge Distillation 해보기 

총 9 가지 조합을 사용 하여 해보기
- Temperature = {1, 20, 50}
- Alpha = {0.2, 0.5, 0.8}


In [ ]:
# Temperature: 1 &  Alpha: 0.2
config.KD.ON_OFF = 'on' # student를 knowledge distillation으로 학습한다면 'on'
config.KD.TEMPERATURE = 1
config.KD.ALPHA = 0.2
main(config)

In [ ]:
# Temperature: 1 &  Alpha: 0.5
config.KD.ON_OFF = 'on' # student를 knowledge distillation으로 학습한다면 'on'
config.KD.TEMPERATURE = 1
config.KD.ALPHA = 0.5
main(config)

In [ ]:
# Temperature: 1 &  Alpha: 0.8
config.KD.ON_OFF = 'on' # student를 knowledge distillation으로 학습한다면 'on'
config.KD.TEMPERATURE = 1
config.KD.ALPHA = 0.8
main(config)